# Annotations of the workflow

The idea is to get more content about the project/workflow.

* Conda environment for this Notebook: [Greference_tools](../../environments/Greference_tools.yml)

In [ ]:
## Import libreries

# Base packages
import subprocess
import os
import re
import random
# Data manipulation
import pandas as pd
import numpy as np

In [ ]:
random.seed(97)

# Metadata/report.csv

Lets see our Project and all the fields.

For this project we will be interested in download the Bam files in the ***8 field***: **submitted_ftp**

In [ ]:
report = pd.read_csv("../../../metadata/report.tsv", sep="\t")
report

## Downloading the table 3, for supporting information found in the article.

After that we will use the function in pandas ```pd.read_clipboard()``` to download the data. 

You have to copy the table and run the function.

https://onlinelibrary.wiley.com/action/downloadSupplement?doi=10.1002%2Fgcc.22314&file=gcc22314-sup-0005-supptable5.docx

In [ ]:
table3=pd.read_clipboard()
table3.head()

In [ ]:
## We wolud do some tranformations to the first field ("Sample")
table3["Sample"] = table3["Sample"].str.replace(
    "-", "_"
    ).str.replace(
        "^", "_method1"
        ).str.replace(
            "*", "_method2"
            )
table3.head()

In [ ]:
table3.to_csv("../../../metadata/table3.csv", index=None)

In [ ]:
table3_csv = pd.read_csv("../../../metadata/table3.csv")
table3_csv

### Amount of reads after removing low quality sequences. 
### Low quality sequences are considered when

1. bases with Q20 account for >85% of the total sequence length 
2. reads with N-rate >10%

### Methods for seguencing

* **method1** run with Agilent Sureselect XT Human All Exon 50Mb
* **method2** run with Agilent SureSelect_XT_Human_All_Exon_V5

## Mean depth of the project

In [ ]:
table3_csv["Sequencing depth (X)"].str.replace(",",".").astype(float).mean()

## **Plot of the rules from the snakemake pipeline**

In [ ]:
subprocess.run(["python", "../scripts_analysis/workflow_viz.py"])

<p align="center">
 
  <img src="../results_analysis/plots/snakemake_rules_linear.png" alt="600" width="500" />

</p>

Plot to show the structure to follow of the pipline

* Color *green*: you have to execute **multiple** times this rule (as much chromosomes/genes you want to filter).

* Color *orange*: you have to execute only **1** time, the first run of the workflow.

* Color *red*: you have to execute only **1** time, the last run, after having all genes filter tables.

## **Calculating the AVERAGE sequence length for each sample**  

In [ ]:
## heavy computing +25 min -> you need all fastq.gz files in data/raw/

for chr in ["chr3", "chr5", "chr7", "chr12", "chr17"]:
    subprocess.run(["bash", "../scripts_analysis/01calculating_length.sh", chr])

subprocess.run(["bash", "../scripts_analysis/02joining_files.sh"])


In [ ]:
list_files = os.listdir("../../../results/annotations/")

list_files_path = []

for file in range(len(list_files)):
    path = "../../../results/annotations/" + list_files[file]
    list_files_path.append(path)

In [ ]:
list_mean_len = []
list_sd_len = []

for file_path in list_files_path:

    with open(file_path) as file:
        len_list = file.readlines()
    
    mean_seq_len = np.array([s.strip() for s in len_list]).astype(int).mean().round(2)
    sd_seq_len = np.array([s.strip() for s in len_list]).astype(int).std().round(2)
    
    print("mean value of ", file_path.replace(
        "../../../results/annotations/", ""
        ).replace(
            "_len.txt", ""
            ), " -> ", mean_seq_len, "+/-", sd_seq_len)

    list_mean_len.append(mean_seq_len)
    list_sd_len.append(sd_seq_len)


In [ ]:
df_len_seq = pd.DataFrame({
    "sample": pd.Series(list_files_path),
    "mean_len": pd.Series(list_mean_len),
    "sd_len": pd.Series(list_sd_len)
})

df_len_seq["sample"] = df_len_seq["sample"].str.replace("../../results/annotations/", "")
df_len_seq["sample"] = df_len_seq["sample"].str.replace("_len.txt", "")

df_len_seq.to_csv("../results_analysis/tables/len_mean.csv", index=None)

In [ ]:
subprocess.run(["Rscript", "../scripts_analysis/03plot_mean_len.R"])

## **Number of Seqs before and after fastp**

* In order to run the next code, you have to have all fastq files (**all chromosomes, reads**) in *raw* and *processed*

In [ ]:
subprocess.run(["bash", "../scripts_analysis/04fastp_analysis.sh"])

In [ ]:
df_fastp = pd.read_csv("../results_analysis/tables/fastp_analysis.tsv", sep="\t")
df_fastp

## Mapped reads, with the reference Genome

* ### flagstats: in order to do this analysis, you need all the flagstats information in metadata/logs/flagstats.

In [ ]:
subprocess.run(["bash", "../scripts_analysis/05flagstats.sh"])

In [ ]:
flagstats = pd.read_csv("../results_analysis/tables/flagstats.tsv", sep="\t")
flagstats["mapped_reads_per"] = np.array(flagstats.mapped_reads/flagstats.num_seqs_afterQC*100).round(2)
flagstats["properly_paired_per"] = np.array(flagstats.properly_mapped/flagstats.num_seqs_afterQC*100).round(2) 
flagstats["singletons_per"] = np.array(flagstats.singletons/flagstats.num_seqs_afterQC*100).round(2)  

In [ ]:
flagstats.to_csv("../results_analysis/tables/flagstats.tsv", sep="\t")

In [ ]:
flagstats

### Plotting the previous table

In [ ]:
subprocess.run(["Rscript", "../scripts_analysis/06plot_flagstats.R"])

## VCF stats

* For this analysis you'll need all the files created in the dir metadata/logs/vcfstats  

In [ ]:
subprocess.run(["bash", "../scripts_analysis/07vcfstats.sh"])

In [ ]:
## Process additional vcf stats 
vcf_stats = pd.read_csv("../results_analysis/tables/additional_vcf_stats.tsv", sep="\t", index_col=False)
vcf_stats

In [ ]:
new_cols = []
for col in vcf_stats.iloc[:,:9].columns:
    new_cols.append("r_" + col)

print(new_cols)

for name,ncol in zip(new_cols, range(len(vcf_stats.columns) -1)):
    vcf_stats[name] = vcf_stats.iloc[:,ncol].str.split("|").apply(lambda x: float(x[0]) / float(x[1])).round(2)

vcf_stats

ratios_vstats = vcf_stats.iloc[:,9:]

In [ ]:
for name in vcf_stats.iloc[:,:9]:
    vcf_stats[name] = "(" + vcf_stats[name].str.replace("|", "/") + ")"
    
for name,r_name in zip(vcf_stats.iloc[:,:9].columns, vcf_stats.iloc[:,10:].columns):
    vcf_stats[name] = vcf_stats[r_name].astype(str) + " " + vcf_stats[name]
    

In [ ]:
vcf_stats = vcf_stats.iloc[:,:10]

vcf_stats.to_csv("../results_analysis/tables/additional_vcf_stats.tsv")


In [ ]:
vcf_stats

In [ ]:
ratios_vstats.to_csv("../results_analysis/tables/ratios_vcfstats.tsv", sep="\t")
ratios_vstats.head()

In [ ]:
subprocess.run(["Rscript", "../scripts_analysis/08ploting_vcfstats.R"])

In [ ]:
## Statisticall analysis from the last plot
subprocess.run(["Rscript", "../scripts_analysis/09statistical_analysis.R"])